In [2]:
import pandas as pd
import altair as alt

In [3]:
samples=["I15566-L1","115_L001","139_L001"]
cont_warning= 50

sum_dict={}
for sample in samples:
    sample_sum_dict = {}
    stats_path=f"../../results/23_12_18/contamination/stats_{sample}.txt"
    with open(stats_path, "r") as stats:
        for line in stats:
            if line.startswith("SN	sequences"):
                total=line.split(":")[-1].strip()
                continue
            elif line.startswith("SN	reads mapped"):
                mapped=line.split(":")[-1].strip()
                sample_sum_dict["reads_mapped"] =int(mapped)

                prc=(int(mapped)/int(total))
                sample_sum_dict["contamination"] = "%.6f" % prc

                if prc >= cont_warning:
                    sample_sum_dict["color"] = "#9A0430"
                else:
                    sample_sum_dict["color"] = "#6ea165"

                break

    sum_dict[sample]=sample_sum_dict

sum_df = pd.DataFrame.from_dict(sum_dict, orient="index")
#sum_df.index.name = "sample"
#sum_df.insert(0,'sample','')
#sum_df['sample'] = sum_df.index
sum_df = sum_df.reset_index()
sum_df.rename(columns={"index":"sample"}, inplace=True)
print(sum_df)

      sample  reads_mapped contamination    color
0  I15566-L1        107793      0.005899  #6ea165
1   115_L001          1068      0.000051  #6ea165
2   139_L001           260      0.000013  #6ea165


In [20]:
slider = alt.binding_range(min=0, max=100, step=0.5, name='max human contamination')
selector = alt.param(name='SelectorName', value=50, bind=slider)


base_chart=alt.Chart(sum_df,title="% human contamination").encode(
    alt.X('contamination:Q').axis(format='%',labelFontSize=12, titleFontSize=15).title('human contamination'),
    alt.Y('sample:N').axis(labelFontSize=12, titleFontSize=15),
).add_params(
   selector
).properties(width=600).interactive()


bars=base_chart.mark_bar().encode(color=alt.condition(
       alt.datum.contamination >= selector,
       alt.value('#9A0430'),
       alt.value('#6ea165')
   ))


chart_text = base_chart.mark_text(
    align='center',
    baseline='middle',
    dx=20,
    fontSize=12,
).encode(
    text=alt.Text("contamination:Q",format='.2%'),)
    #text='human contamination (%):Q')
#chart.mark_text(align='left', dx=2)
charty =bars + chart_text
jchart = alt.JupyterChart(charty)
jchart


SchemaValidationError: Multiple errors were found.

Error 1: `LayerChart` has no parameter named 'fontSize'

    Existing parameter names are:
    layer        data          height    projection   usermeta   
    autosize     datasets      name      resolve      view       
    background   description   padding   title        width      
    config       encoding      params    transform               

    See the help for `LayerChart` to read the full description of these parameters

Error 2: `LayerChart` has no parameter named 'fontSize'

    Existing parameter names are:
    layer        data          height    projection   usermeta   
    autosize     datasets      name      resolve      view       
    background   description   padding   title        width      
    config       encoding      params    transform               

    See the help for `LayerChart` to read the full description of these parameters

In [ ]:
chart.save('chart.html')